In [ ]:
import subprocess
import os
import zipfile
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureLayer, FeatureCollection, FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.geometry import Geometry, Point, Polyline, lengths
import csv
from collections import defaultdict
from pykml import parser
from lxml import etree
import os

In [ ]:
# Method 2: Logging in with a client ID
gis_url= "https://your_org.maps.arcgis.com" # url to your org's arcGIS
your_client_id = "your_id"               # paste client ID here
gis = GIS(gis_url, client_id=your_client_id)
print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
# Retrieves the number of buildings, along with each of their latitudess and longitudes, within each of your buffers

buffers = gis.content.search(
    f"title:Idaho,Buffer, 200 AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

# Group buffer features by title, then vert_x, vert_y, tot_score because these three attrbitutes are the same for a pair of x and y arms
buffer_title_groups = defaultdict(lambda: defaultdict(list))
summary = []
results = []
zero_buildings = []
zero_building_pairs = []

# Organize features into dict of vert_x, vert_y, and tot_score
for buffer in buffers:
    input_buffer = buffer.layers[0]
    buffer_features = input_buffer.query(
        where="1=1", out_fields="*", return_geometry=True
    ).features

    for feature in buffer_features:
        attrs = feature.attributes
        buffer_title = buffer.title
        key = (attrs["vert_x"], attrs["vert_y"], attrs["tot_score"])
        feature.attributes["buffer_title"] = buffer_title  
        buffer_title_groups[buffer_title][key].append(feature)

for buffer_title, feature_groups in buffer_title_groups.items():
    site_index = 1 

    for key, feature_list in feature_groups.items():
        if len(feature_list) != 2:
            print(f"More than a pair of arms ({len(feature_list)}) detected for key {key} in buffer '{buffer_title}'")
            continue

        total_count = 0
        all_buildings = []

        for feature in feature_list:
            buffer_geom = feature.geometry
            buildings_clipped = buildings.query(
                geometry_filter=intersects(buffer_geom),
                return_geometry=True
            )
            count = len(buildings_clipped.features)
            total_count += count
            all_buildings.extend(buildings_clipped.features)

        site_name = f"{site_index}_{int(key[2])}_{buffer_title[:-7]}"
        site_index += 1
        summary.append([site_name, total_count])
        if total_count == 0:
            print(f"{site_name}: 0")
            zero_building_pairs.append({
                "vert_x": key[0],
                "vert_y": key[1],
                "tot_score": key[2],
            })
            results.append([site_name, total_count, None, None, None])
            zero_buildings.append([site_name, total_count])
        else:
            for b in all_buildings:
                results.append([site_name,total_count,b.attributes.get("BUILD_ID"),b.attributes.get("LONGITUDE"),b.attributes.get("LATITUDE")])
            print(f"{site_name}: {total_count} building(s)")



In [ ]:
# outputs csv for just building counts, building counts with coordinates, and sites with no buildings

output_summary_name = "Idaho_building_counts_200m.csv"
output_detailed_name = "Idaho_building_coordinates_200m.csv"
output_zero_name = "Idaho_no_buildings_200m.csv"

output_csv_summary = r"C:\Users\tooba\Downloads\Idaho_building_counts_200m.csv"
output_csv_detailed = r"C:\Users\tooba\Downloads\Idaho_building_coordinates_200m.csv"
output_csv_zero = r"C:\Users\tooba\Downloads\Idaho_no_buildings_200m.csv"

with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)

print(f"CSV saved to {os.path.abspath(output_csv_summary)}")


with open(output_csv_detailed, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings","Building ID", "Longitude", "Latitude"])
    writer.writerows(results)

print(f"CSV saved to {os.path.abspath(output_csv_detailed)}")

with open(output_csv_zero, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(zero_buildings)

print(f"CSV saved to {os.path.abspath(output_csv_zero)}")


In [ ]:
reference_dicts = zero_building_pairs
reference_set = {(d["vert_x"], d["vert_y"], d["tot_score"]) for d in reference_dicts}
input_folder = r"C:\Users\tooba\Downloads\IdahoFalls"
output_folder = r"C:\Users\tooba\Downloads\IdahoFallsZeroBuildings"
os.makedirs(output_folder, exist_ok=True)
kml_ns = "{http://www.opengis.net/kml/2.2}"

sites_per_kml = []

def matches_reference(vx, vy, ts, ref_set, tol=1e-6):
    for rvx, rvy, rts in ref_set:
        if abs(vx - rvx) < tol and abs(vy - rvy) < tol and abs(ts - rts) < tol:
            return True
    return False

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".kml"):
        kml_path = os.path.join(input_folder, filename)
        with open(kml_path) as f:
            doc = parser.parse(f).getroot()
            placemarks = list(doc.findall(f".//{kml_ns}Placemark"))
            total_sites = int(len(placemarks) / 2)

            matched_count = 0
           
            for placemark in list(doc.findall(f".//{kml_ns}Placemark")):
                try:
                    simple_data = placemark.findall(f".//{kml_ns}SimpleData")
                    data_dict = {sd.get('name'): sd.text for sd in simple_data}
                    vx = float(data_dict.get('vert_x', 'nan'))
                    vy = float(data_dict.get('vert_y', 'nan'))
                    ts = float(data_dict.get('tot_score', 'nan'))

                    if not matches_reference(vx, vy, ts, reference_set):
                        placemark.getparent().remove(placemark)
                    if matches_reference(vx, vy, ts, reference_set):
                        matched_count += 1
                except Exception:
                    continue
                    
            sites_nb = int(matched_count / 2)
           
            sites_per_kml.append([filename, total_sites, sites_nb])
       
        modified_name = f"{filename[:-4]}" + "_zero_buildings_200m.kml"
        output_path = os.path.join(output_folder, modified_name)
        with open(output_path, "wb") as f:
            f.write(etree.tostring(doc, pretty_print=True, xml_declaration=True, encoding="UTF-8"))
       
        print(f"Saved KML with no buildings to to {output_path}")

output_kml_summary = r"C:\Users\tooba\Downloads\Idaho_sorted_by_buildings_200m.csv"

# Write summary CSV
with open(output_kml_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["KML Name", "Total sites", "Number of sites with no buildings"])
    writer.writerows(sites_per_kml)

print(f"Summary CSV saved to: {os.path.abspath(output_kml_summary)}")
